In [1]:
import sympy as sp
import IPython.display as display
import nbconvert as nbc

# CV: Measurement contains only measured value (without velocity)

In [2]:
x, v, x_meas, dt = sp.symbols('x V x_meas dt')

r_x = sp.symbols('r_x') # R

q_a = sp.symbols('q_a') # Q

p11, p12, p21, p22 \
    = sp.symbols("p_11 p_12 p_21 p_22")

k11, k12 \
    = sp.symbols("k_11 k_12")

I = sp.Matrix([[ 1, 0],
               [ 0, 1]])

In [3]:
X = sp.Matrix([[  x  ],
               [  v  ]])

Z = sp.Matrix([[ x_meas ]])

H = sp.Matrix([[ 1, 0 ]])

F = sp.Matrix([[ 1, dt ],
               [ 0,  1 ]])

G = sp.Matrix([[ dt**2/2 ],
               [   dt    ] ])

R = sp.Matrix([[ r_x ]])

P = sp.Matrix([[ p11, p12 ],
               [ p21, p22 ]])

Q = G * G.T * q_a
Q

Matrix([
[dt**4*q_a/4, dt**3*q_a/2],
[dt**3*q_a/2,   dt**2*q_a]])

In [4]:
P

Matrix([
[p_11, p_12],
[p_21, p_22]])

In [5]:
R

Matrix([[r_x]])

In [6]:
X

Matrix([
[x],
[V]])

In [7]:
Z

Matrix([[x_meas]])

In [8]:
H

Matrix([[1, 0]])

# Prediction

In [9]:
X = F @ X
X

Matrix([
[V*dt + x],
[       V]])

In [10]:
P = F @ P @ F.T + Q
P

Matrix([
[dt**4*q_a/4 + dt*p_21 + dt*(dt*p_22 + p_12) + p_11, dt**3*q_a/2 + dt*p_22 + p_12],
[                      dt**3*q_a/2 + dt*p_22 + p_21,             dt**2*q_a + p_22]])

In [11]:
sp.Eq(p11, P[0,0])

Eq(p_11, dt**4*q_a/4 + dt*p_21 + dt*(dt*p_22 + p_12) + p_11)

In [12]:
sp.Eq(p12, P[0,1])

Eq(p_12, dt**3*q_a/2 + dt*p_22 + p_12)

In [13]:
sp.Eq(p21, P[1,0])

Eq(p_21, dt**3*q_a/2 + dt*p_22 + p_21)

In [14]:
sp.Eq(p22, P[1,1])

Eq(p_22, dt**2*q_a + p_22)

In [15]:
X = sp.Matrix([[  x  ],
               [  v  ]])
X

Matrix([
[x],
[V]])

In [16]:
P = sp.Matrix([[ p11, p12 ],
               [ p21, p22 ]])
P

Matrix([
[p_11, p_12],
[p_21, p_22]])

# Correction

In [17]:
Y = Z - (H @ X)
Y

Matrix([[-x + x_meas]])

In [18]:
S = (H @ P @ H.T) + R
S

Matrix([[p_11 + r_x]])

In [19]:
K = P @ H.T @ S.inv()
K

Matrix([
[p_11/(p_11 + r_x)],
[p_21/(p_11 + r_x)]])

In [20]:
sp.Eq(k11, K[0,0])

Eq(k_11, p_11/(p_11 + r_x))

In [21]:
sp.Eq(k12, K[1,0])

Eq(k_12, p_21/(p_11 + r_x))

In [22]:
K = sp.Matrix([[ k11 ],
               [ k12 ]])
K

Matrix([
[k_11],
[k_12]])

In [23]:
X = X + (K @ Y)
X

Matrix([
[k_11*(-x + x_meas) + x],
[V + k_12*(-x + x_meas)]])

In [24]:
sp.Eq(x, X[0,0])

Eq(x, k_11*(-x + x_meas) + x)

In [25]:
sp.Eq(v, X[1,0])

Eq(V, V + k_12*(-x + x_meas))

In [26]:
P = (I - (K @ H)) @ P
P

Matrix([
[  p_11*(1 - k_11),   p_12*(1 - k_11)],
[-k_12*p_11 + p_21, -k_12*p_12 + p_22]])

In [27]:
sp.Eq(p11, P[0,0])

Eq(p_11, p_11*(1 - k_11))

In [28]:
sp.Eq(p12, P[0,1])

Eq(p_12, p_12*(1 - k_11))

In [29]:
sp.Eq(p21, P[1,0])

Eq(p_21, -k_12*p_11 + p_21)

In [30]:
sp.Eq(p22, P[1,1])

Eq(p_22, -k_12*p_12 + p_22)